<a href="https://colab.research.google.com/github/GuJiMao/GNN_Learning/blob/main/%E5%87%86%E5%A4%87%E8%87%AA%E5%B7%B1%E7%9A%84%E6%95%B0%E6%8D%AE%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 说明
[原文链接](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_dataset.html)

# 创建自己的数据集

尽管PyTorch Geometric已经包含了大量有用的数据集，但您可能希望使用自己记录的或非公开可用的数据创建自己的数据集。

自己实现数据集很简单，您可能想看看源代码，了解各种数据集是如何实现的。但是，我们将简要介绍如何设置您自己的数据集。

我们为数据集提供了两个抽象的类：`torch_geometric.data.Dataset`和`torch_geometric.data.InMemoryDataset`。`torch_geometirc.data.InMemoryDataset`继承自`torch_geometric.data.Dataset`，如果整个数据在CPU内存上的话就应该使用这个类。

按照`torchvision`约定，每个数据集都会被传递一个根文件夹，它指示数据集应该存储在哪里。我们将根文件夹拆分为两个文件夹:`raw_dir`(数据集下载到其中)和`processed_dir`(已处理数据集保存在其中)。

此外，可以给每个数据集传递一个`transform`、一个`pre_transform`和一个`pre_filter`函数，这些函数默认为`None`。`transfor`m函数在访问数据对象之前动态地转换数据对象(因此最适合用于数据扩展)。`pre_transform`函数在将数据对象保存到磁盘之前应用转换(因此，它最适合于只需要执行一次的繁重预计算)。`pre_filter`函数可以在保存数据之前手动过滤掉数据对象。用例可能涉及数据对象是特定类的限制。

## 创建`InMemoryDatasets`

为了创建一个`torch_geometric.data.InMemoryDataset`，需要实时四个基础的方法：
- `torch_geometric.data.InMemoryDataset.raw_file_names`：在`raw_dir`中的一系列文件
- `torch_geometric.data.InMemoryDataset.processed_file_names`：在`processed_dir`中的一些列文件，这些文件避免再次处理
- `torch_geometric.data.InMemoryDataset.download()`：下载数据到`raw_dir`中
- `torch_geometric.data.InMemoryDataset.process()`：处理原始数据并将它保存到`processed_dir`路径中去

在`torch_geometric.data`中能找到有用的方法来下载和提取数据。

最主要的部分在`process()`这个函数中。在这，我们需要读取和创建一些列的`Data`对象，然后将它保存到`processed_dir`路径中。因为保存一个很大的python列表是很慢的，所以在保存前使用`torch_geometric.data.InMemoryDataset.collate()`函数将Python列表放到一个大的`Data`对象中。这个`Data`对象将所有的的例子中的对象放到了一起，并且返回了一个`slices`路径来从这个`Data`对象中抽取一个单个对象。最后，我们需要将`self.data`和`self.slices`这两个属性加载到这两个对象中去。

下面有一个简单的例子：


In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url

class MyOwnDataset(InMemoryDataset):
  def __init__(self, root, transform=None, pre_transform=None):
    super(MyOwnDataset, self).__init__(root, transform, pre_transform)
    self.data, self.slices = torch.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return ['some_file_1', 'some_file_2']

  @property
  def processed_file_names(self):
    return['data.pt']

  def download(self):
    # 下载到self.raw_dir中
    download_url(url, self.raw_dir)
    ...
  def process(self):
    # 读取数据到Data列表中
    data_list = [...]

    if self.pre_filter is not None:
      data_list = [data for data in data_list if self.pre_filter(data)]

    if self.pre_transform is not None:
      data_list = [self.pre_transform(data) for data in data_list]

    data, slices = self.collate(data_list)
    torch.save((data, slices), self.processed_paths[0])

## 创建更大的数据集

对于创建不适合内存的数据集，可以使用`torch_geometry .data`。可以使用数据集，它紧跟着`torchvision`数据集的概念。它还期望实现以下方法:
- `torch_geometric.data.Dataset.len()`：返回数据集中的例子的个数
- `torch_geometirc.data.Dataset.get()`：实现加载单个图的逻辑

在内部，`torch_geometric.data.Dataset .__ getitem __()`从`torch_geometric.data.Dataset.get()`获取数据对象，并根据需要对它们进行变换。

下面看一个简单的例子

In [ ]:
import os.path as osp

import torch
from torch_geometric.data import Dataset, download_url


class MyOwnDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyOwnDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data_1.pt', 'data_2.pt', ...]

    def download(self):
        # Download to `self.raw_dir`.
        path = download_url(url, self.raw_dir)
        ...

    def process(self):
        i = 0
        for raw_path in self.raw_paths:
            # Read data from `raw_path`.
            data = Data(...)

            if self.pre_filter is not None and not self.pre_filter(data):
                continue

            if self.pre_transform is not None:
                data = self.pre_transform(data)

            torch.save(data, osp.join(self.processed_dir, 'data_{}.pt'.format(i)))
            i += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, 'data_{}.pt'.format(idx)))
        return data

在这过程中，图数据对象在`process()`过程中就已经保存了，需要手动加载到`get()`

# 常见问题

## 如何跳过`download()`或者`process()`过程
可以通过重载`download()`和`process()`方法来跳过这个过程

In [ ]:
class MyOwnDataset(Dataset):
    def __init__(self, transform=None, pre_transform=None):
        super(MyOwnDataset, self).__init__(None, transform, pre_transform)

## 是否必须用到这些数据集接口
不！ 就像在常规PyTorch中一样，您无需使用数据集，例如，当您想动态创建合成数据而无需将其显式保存到磁盘时。 在这种情况下，只需传递一个包含`torch_geometric.data.Data`对象的常规python列表，然后将它们传递到`torch_geometric.data.DataLoader`：

In [ ]:
from torch_geometric.data import Data, DataLoader

data_list = [Data(...), ..., Data(...)]
loader = DataLoader(data_list, batch_size=32)